# Lista de Exercícios - Aprendizagem de Máquina

Aluna: Bruno Cardoso Dantas

### Questão 01

Usando a base de dados Porto Seguro’s Safe Driver Prediction¹ para estimar quando um cliente irá acionar o seguro nos próximos 12 meses, pede-se:

*   (a) Gere bases de treinamento, validação, e teste balanceadas realizando sub-amostragem da classe mais numerosa;

*   (b) Considerando os atributos binários da base de dados como categóricos, avalie:

i. as taxas de acerto obtidas na base de teste com Regressão Logística e K-vizinhos mais próximos considerando apenas os atributos numéricos;

ii. as taxas de acerto obtidas na base de teste com Naive Bayes e Árvore de
Decisão considerando apenas os atributos categóricos;

iii. a taxa de acerto obtida com Random Forest na base de teste considerando
todos os atrbutos da base de dados;

iv. Quais as taxas de erro em (i), (ii), e (iii) considerando intervalo de confiança e confiabilidade de 90%?

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls

drive  sample_data


In [4]:
%cd /content/drive/MyDrive/UFPE - Ciência de Dados/APRENDIZAGEM DE MÁQUINA/Dataset/porto/

!ls

/content/drive/MyDrive/UFPE - Ciência de Dados/APRENDIZAGEM DE MÁQUINA/Dataset/porto
sample_submission.csv  test.csv  train.csv


In [5]:
# Carregar os dados

df_test = pd.read_csv('./test.csv')
df_train = pd.read_csv('./train.csv')
df_submission = pd.read_csv('./sample_submission.csv')

In [6]:
# Dividir o conjunto de treinamento original em treinamento (80%) e validação (20%)

df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=42)

In [7]:
# Separar as classes majoritárias e minoritárias

majority_class = df_train[df_train['target'] == 0]
minority_class = df_train[df_train['target'] == 1]

In [8]:
# Realizar sub-amostragem da classe majoritária para equilibrar as classes

majority_downsampled = resample(majority_class, replace=False, n_samples=len(minority_class), random_state=42)

In [9]:
# Criar o novo conjunto de treinamento balanceado

df_train_balanced = pd.concat([majority_downsampled, minority_class])
df_train_balanced = df_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_validation = df_validation.sample(frac=1, random_state=42).reset_index(drop=True)
df_test = df_test.sample(frac=1, random_state=42).reset_index(drop=True)

In [10]:
# Identificar atributos categóricos e numéricos com base nos sufixos

categorical_features = [col for col in df_train.columns if col.endswith(('_bin', '_cat'))]
numeric_features = [col for col in df_train.columns if col not in categorical_features and col != 'target']

In [11]:
# Filtrar apenas atributos numéricos para treinamento e teste

X_train_numeric = df_train[numeric_features]
X_test_numeric = df_test[numeric_features]

In [12]:
# Normalizar os atributos numéricos (opcional, mas muitas vezes útil)

scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

In [13]:
# Treinar modelo de Regressão Logística
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(X_train_numeric_scaled, df_train['target'])

LogisticRegression(random_state=42)

In [14]:
# Fazer previsões na base de teste
logistic_predictions = logistic_model.predict(X_test_numeric_scaled)

In [15]:
# Exibir as previsões
df_predictions = pd.DataFrame({'id': df_test['id'], 'target': logistic_predictions})
print(df_predictions)

             id  target
0        820692       0
1        587380       0
2        668609       0
3        507084       0
4       1460105       0
...         ...     ...
892811   432065       0
892812   610085       0
892813   219775       0
892814  1118843       0
892815   203232       0

[892816 rows x 2 columns]


In [16]:
# Treinar modelo K-vizinhos mais próximos
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_numeric_scaled, df_train['target'])

KNeighborsClassifier()

In [ ]:
# Fazer previsões na base de teste
knn_predictions = knn_model.predict(X_test_numeric_scaled)

In [ ]:
# Exibir as previsões (opcional)
df_predictions_knn = pd.DataFrame({'id': df_test['id'], 'prediction': knn_predictions})
print(df_predictions_knn)

In [ ]:
# Avaliar a acurácia do modelo de Regressão Logística
accuracy_logistic = accuracy_score(df_test['target'], logistic_predictions)
print(f'Acurácia do modelo de Regressão Logística: {accuracy_logistic:.4f}')

In [ ]:
# Avaliar a acurácia do modelo K-vizinhos mais próximos
accuracy_knn = accuracy_score(df_test['target'], knn_predictions)
print(f'Acurácia do modelo K-vizinhos mais próximos: {accuracy_knn:.4f}')